Installations:

In [ ]:
!nvidia-smi

Fri May  2 12:57:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sudo sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!nohup ollama serve > /dev/null 2>&1 &

In [ ]:
!pwd

/content


In [ ]:
models = [
    "llama3",
    "deepseek-r1",
    "deepseek-r1:1.5b",
    "qwen2.5",
    "qwen2.5:1.5b",
    "gemma3:1b",
    "gemma3"
]

Running Services in Background:

In [ ]:
!ollama pull llama3
!ollama pull deepseek-r1
!ollama pull deepseek-r1:1.5b
!ollama pull qwen2.5
!ollama pull qwen2.5:1.5b
!ollama pull gemma3:1b
!ollama pull gemma3

Streaming output truncated to the last 5000 lines.
pulling 6a0746a1ec1a: 100% ▕▏ 4.7 GB                         
pulling 4fa551d4f938: 100% ▕▏  12 KB                         
pulling 8ab4849b038c: 100% ▕▏  254 B                         
pulling 577073ffcc6c: 100% ▕▏  110 B                         pulling manifest 
pulling 6a0746a1ec1a: 100% ▕▏ 4.7 GB                         
pulling 4fa551d4f938: 100% ▕▏  12 KB                         
pulling 8ab4849b038c: 100% ▕▏  254 B                         
pulling 577073ffcc6c: 100% ▕▏  110 B                         pulling manifest 
pulling 6a0746a1ec1a: 100% ▕▏ 4.7 GB                         
pulling 4fa551d4f938: 100% ▕▏  12 KB                         
pulling 8ab4849b038c: 100% ▕▏  254 B                         
pulling 577073ffcc6c: 100% ▕▏  110 B                         pulling manifest 
pulling 6a0746a1ec1a: 100% ▕▏ 4.7 GB                         
pulling 4fa551d4f938: 100% ▕▏  12 KB                         
pulling 8ab4849b038c: 100% ▕▏ 

Create Required Directories:

In [ ]:
import os
os.makedirs("data", exist_ok=True)
os.makedirs("result", exist_ok=True)


Read data:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Initialize Values:

In [ ]:
messages = []
prompt_template_symbol = "Extract cryptocurrency symbols mentioned in the text (e.g., BTC, ETH, BCH). Return only the symbol names.\n\nText: "
prompt_template_timespan = "Extract the timespan from the text: Short term, Mid term, Long term, or Not mentioned. Return only the label.\n\nText: "
prompt_template_sentiment = "Extract the overall sentiment toward the symbol in the text. Return only: POSITIVE, NEGATIVE, or NEUTRAL.\n\nText: "

Read Json Messages

In [ ]:
import json
import random
def process_messages ():
  for i in range(1, 4):
    with open(f"/content/drive/MyDrive/telegram/{i}.json", 'r+') as f:
        chat = json.load(f)
        print(len(chat["messages"]))
        for item in chat["messages"]:
            text = "".join(entity["text"] for entity in item["text_entities"])
            if len(text) < 6: continue
            messages.append({
                "timestamp": item["date"],
                "text": text
            })

Setup Database Connection:

In [ ]:
import sqlite3
def create_connection():
    conn = None
    conn = sqlite3.connect("result/models-without-fine-tuning.db")
    print(f"Connected to SQLite version {sqlite3.version}")
    return conn

def create_table(conn):
    c = conn.cursor()
    statement = """CREATE TABLE IF NOT EXISTS results (
                id integer PRIMARY KEY,
                message text,"""
    for model in models:
      statement = statement + f"""{model}_symbols,
          {model}_timespan,
          {model}_sentiment,"""
    statement = statement + "timestamp text NOT NULL)"
    c.execute(statement.replace("-","_").replace(":","_").replace(".", "_"))

def insert_data(conn, data):
    statement = f"""INSERT INTO results(timestamp,message"""
    for model in models:
      statement = statement + f",{model}_symbols,{model}_timespan,{model}_sentiment"
    statement = statement + f""")
        VALUES(?,?"""
    for model in models:
            statement = statement + f",?,?,?"
    statement = statement + ") "
    # sql = f"""INSERT INTO results(timestamp,start_timestamp,end_timestamp,text,symbols,timespan,sentiment)
    #           VALUES(?,?,?,?,?,?,?) """
    cur = conn.cursor()
    cur.execute(statement.replace("-","_").replace(":","_").replace(".", "_"), data)
    conn.commit()
    return cur.lastrowid

Send Message To Model

In [ ]:
!pip install ollama

In [ ]:
# import datetime
# from time import sleep
# if __name__ == "__main__":
#   process_messages()
#   conn = create_connection()
#   create_table(conn)
#   index = 0
#   for item in messages:
#     index = index+1
#     print(index)
#     # start_time = datetime.datetime.now().timestamp()
#     data = [item["timestamp"]]
#     for model in models:
#       symbol_prompt = prompt_template_symbol + item["text"]
#       symbol_respone = send_to_llm(symbol_prompt, model)
#       timespan_prompt = prompt_template_timespan + item["text"]
#       timespan_respone = send_to_llm(timespan_prompt, model)
#       sentiment_prompt = prompt_template_sentiment + item["text"]
#       sentiment_respone = send_to_llm(sentiment_prompt, model)
#       symbol = clean_ollama_response(symbol_respone)
#       timespan = clean_ollama_response(timespan_respone)
#       sentiment = clean_ollama_response(sentiment_respone)
#       end_time = datetime.datetime.now().timestamp()
#       data.append(symbol)
#       data.append(timespan)
#       data.append(sentiment)
#     insert_data(conn, tuple(data))
#     sleep(0.1)


In [ ]:
# from ollama import chat
# import re
# def clean_ollama_response(response_text):
#     # Remove <think>...</think> parts (including any text inside them)
#     cleaned_text = re.sub(r'<think>.*?</think>', '', response_text, flags=re.DOTALL)

#     # Remove extra whitespace (optional)
#     cleaned_text = cleaned_text.strip()

#     return cleaned_text

# def send_to_llm(message, model):
#     response: ChatResponse = chat(model=model, messages=[
#     {
#         'role': 'user',
#         'content': message,
#     },
#     ])
#     return response['message']['content']


Run Main Thread:

In [ ]:
import datetime
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict, Any, Tuple
from ollama import chat

# Pre-compile regex patterns for better performance
THINK_PATTERN = re.compile(r'<think>.*?</think>', flags=re.DOTALL)

def clean_ollama_response(response_text: str) -> str:
    """Optimized response cleaner with pre-compiled regex"""
    return THINK_PATTERN.sub('', response_text).strip()

def send_to_llm(message: str, model: str) -> str:
    """Send message to LLM and return cleaned response"""
    response = chat(model=model, messages=[{'role': 'user', 'content': message}])
    return clean_ollama_response(response['message']['content'])

In [ ]:
def process_model_prompts(model: str, text: str) -> Tuple[str, str, str]:
    """Process all prompts for a single model in parallel"""
    # Create executor outside the with block to reuse it
    executor = ThreadPoolExecutor(max_workers=3)  # One for each prompt type

    try:
        # Submit all prompt tasks
        future_symbol = executor.submit(
            send_to_llm, prompt_template_symbol + text, model
        )
        future_timespan = executor.submit(
            send_to_llm, prompt_template_timespan + text, model
        )
        future_sentiment = executor.submit(
            send_to_llm, prompt_template_sentiment + text, model
        )

        # Wait for all futures to complete with error handling
        symbol = future_symbol.result()
        timespan = future_timespan.result()
        sentiment = future_sentiment.result()

        return symbol, timespan, sentiment

    except Exception as e:
        print(f"Error processing model {model}: {str(e)}")
        # Return default values or re-raise as needed
        return "ERROR", "ERROR", "ERROR"

    finally:
        executor.shutdown(wait=True)

def process_item(item: Dict[str, Any], models: List[str]) -> List[str]:
    """Process a single item across all models"""
    text = item["text"]
    data = [item["timestamp"], text]

    for model in models:
        symbol, timespan, sentiment = process_model_prompts(model, text)
        data.extend([symbol, timespan, sentiment])

    return data

def main_parallel_processing():
    """Main function with parallel processing"""
    # Initialize your variables (these should be defined elsewhere)
    process_messages()
    samples = random.sample(messages, 1000)
    print(len(samples))
    # Initialize database connection
    conn = create_connection()
    create_table(conn)

    # Process all messages - consider adjusting max_workers based on your system
    with ThreadPoolExecutor(max_workers=4) as executor:
        # Submit all message processing tasks
        future_to_index = {
            executor.submit(process_item, item, models): idx
            for idx, item in enumerate(samples, 1)
        }

        # Process completed tasks as they finish
        for future in as_completed(future_to_index):
            idx = future_to_index[future]
            try:
                data = future.result()
                insert_data(conn, tuple(data))
                print(f"Completed processing message {idx}")
            except Exception as e:
                print(f"Error processing message {idx}: {str(e)}")

    # Close database connection
    conn.close()

if __name__ == "__main__":
    main_parallel_processing()

15090
1467
3835
1000
Connected to SQLite version 2.6.0
Completed processing message 4
Completed processing message 2
Completed processing message 3
Completed processing message 1
Completed processing message 5
Completed processing message 6
Completed processing message 7
Completed processing message 8
Completed processing message 10
Completed processing message 9
Completed processing message 11
Completed processing message 12
Completed processing message 13
Completed processing message 14
Completed processing message 15
Completed processing message 16
Completed processing message 17
Completed processing message 18
Completed processing message 20
Completed processing message 19
Completed processing message 21
Completed processing message 22
Completed processing message 23
Completed processing message 24
Completed processing message 25
Completed processing message 27
Completed processing message 26
Completed processing message 28
Completed processing message 29
Completed processing messa